## Python Script for Well Cluster Analysis (Concise Explanation)

This script analyzes well data from an Excel file to identify spatial clusters based on location using DBSCAN. It then filters these clusters by minimum well count and maximum area, providing detailed information about the valid clusters and the wells within them.

**Key Configuration:**

-   **File Paths:** Input Excel (`excel_file_path`), Output CSV (`output_csv_path`).
-   **Clustering:** Minimum wells per cluster (`MIN_WELLS_IN_CLUSTER`), maximum cluster area in km² (`MAX_CLUSTER_AREA_KM2`), DBSCAN epsilon in km (`EPS_KM`).
-   **Column Names:** Specify the exact names for Latitude (`LAT_COL`), Longitude (`LON_COL`), Interpolated gradient (`GRADIENT_COL`), and BHT (`BHT_COL`).

**Main Steps:**

1.  **Load Data:** Reads well data from the specified Excel file.
2.  **Clean Data:** Removes rows with missing or invalid data in essential columns.
3.  **Cluster Wells:** Uses DBSCAN to group wells based on their geographic proximity.
4.  **Analyze Clusters:**
    -   Filters clusters based on the minimum number of wells.
    -   Calculates the area of each remaining cluster using UTM projection.
    -   Keeps only clusters whose area is within the specified maximum.
5.  **Calculate Zone Statistics:** For valid clusters (zones), calculates the average latitude, longitude, gradient, and BHT.
6.  **Prepare Output:** Creates a CSV file containing detailed information for each well in a valid zone, including a zone ID, zone statistics, and the well's original data.

**Output CSV File:**

The output CSV file will contain:

-   `zone_id`: Unique identifier for each valid cluster.
-   `zone_num_wells`: Number of wells in the zone.
-   `zone_center_latitude`, `zone_center_longitude`: Center coordinates of the zone.
-   `zone_area_km2`: Area of the zone in square kilometers.
-   `zone_average_gradient`, `zone_average_bht`: Average gradient and BHT within the zone.
-   All original columns from the input Excel file for each well.



In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull
from pyproj import Proj, Transformer
import math
import warnings

# --- Configuration ---
# !!! 'Latitude', 'Longitude', 'Interpolated gradient', 'BHT', 'Name', 'Api' (or adjust names below)
excel_file_path = 'wells_with_bht.xlsx'
output_csv_path = 'well_clusters_detailed_analysis.csv' # Changed output filename

# Clustering parameters
MIN_WELLS_IN_CLUSTER = 5 # Corresponds to "5 or more wells" (CHANGED FROM 6)
MAX_CLUSTER_AREA_KM2 = 1.0 # Maximum area constraint
# DBSCAN eps: Maximum distance between samples for one to be considered as in the neighborhood of the other.
# We set it based on the area constraint. A 1km radius circle is ~3.14 km^2. A 1km^2 circle has radius ~0.56km.
# Let's use eps related to ~0.7 km as a starting point (might need tuning).
# Earth radius in kilometers
EARTH_RADIUS_KM = 6371.0
EPS_KM = 0.7 # distance in km
# Convert distance to radians for Haversine metric in DBSCAN
eps_rad = EPS_KM / EARTH_RADIUS_KM

# Columns to use (Adjust these names if they are different in your Excel file)
# Make sure these match your Excel file exactly (case-sensitive)
LAT_COL = 'Latitude'
LON_COL = 'Longitude'
GRADIENT_COL = 'Interpolated gradient' # Or the actual name for gradient
BHT_COL = 'BHT' # Or the actual name for Bottom Hole Temperature
# We don't strictly need Name/API for the logic, but they will be in the output
# NAME_COL = 'Name'
# API_COL = 'Api'

# --- Helper Functions ---

def get_utm_zone(longitude):
    """Estimates the UTM zone number from longitude."""
    return int(math.floor((longitude + 180) / 6) + 1)

def calculate_utm_area(lat_lon_points, lat_col_name, lon_col_name):
    """
    Calculates the area of the convex hull of points in km^2 using UTM projection.
    Args:
        lat_lon_points (pd.DataFrame): DataFrame with latitude and longitude columns.
        lat_col_name (str): Name of the latitude column.
        lon_col_name (str): Name of the longitude column.
    Returns:
        float: Area in square kilometers, or None if calculation fails.
    """
    if len(lat_lon_points) < 3:
        # Convex hull requires at least 3 points. Area is 0 for fewer points or co-linear points.
        return 0.0

    # Use the centroid to determine the UTM zone
    center_lat = lat_lon_points[lat_col_name].mean()
    center_lon = lat_lon_points[lon_col_name].mean()
    utm_zone = get_utm_zone(center_lon)

    # Determine if it's North or South hemisphere
    hemisphere = 'north' if center_lat >= 0 else 'south'

    # Define WGS84 and the target UTM projection
    wgs84 = 'epsg:4326' # Standard Lat/Lon
    utm_crs = f'+proj=utm +zone={utm_zone} +{hemisphere} +ellps=WGS84 +datum=WGS84 +units=m +no_defs'

    try:
        # Create a transformer
        transformer = Transformer.from_crs(wgs84, utm_crs, always_xy=True) # lon, lat order

        # Transform points to UTM coordinates (meters)
        utm_coords = [transformer.transform(lon, lat) for lon, lat in zip(lat_lon_points[lon_col_name], lat_lon_points[lat_col_name])]
        utm_points = np.array(utm_coords)

        # Calculate the convex hull
        hull = ConvexHull(utm_points)

        # Area of the convex hull in square meters
        area_m2 = hull.volume # In 2D, 'volume' gives the area

        # Convert area to square kilometers
        area_km2 = area_m2 / 1_000_000
        return area_km2

    except Exception as e:
        warnings.warn(f"Could not calculate UTM area for cluster around ({center_lat:.4f}, {center_lon:.4f}): {e}")
        # Fallback or error handling: Could return None, 0, or estimate differently
        return None


# --- Main Script ---

print(f"Loading data from: {excel_file_path}")
try:
    df = pd.read_excel(excel_file_path)
    print(f"Loaded {len(df)} wells.")
    print("Original columns:", df.columns.tolist()) # Print column names to help debugging
except FileNotFoundError:
    print(f"ERROR: File not found at {excel_file_path}")
    print("Please ensure the file exists and the path is correct.")
    exit()
except Exception as e:
    print(f"ERROR: Could not read Excel file: {e}")
    exit()

# Basic Data Cleaning / Preparation
# Check required columns exist
required_cols_logic = [LAT_COL, LON_COL, GRADIENT_COL, BHT_COL]
missing_cols = [col for col in required_cols_logic if col not in df.columns]
if missing_cols:
    print(f"ERROR: Missing required columns for calculations: {missing_cols}")
    print(f"Please ensure the columns are named correctly (case-sensitive). Expected: {required_cols_logic}")
    exit()

# Keep track of original columns for final output
original_columns = df.columns.tolist()

# Drop rows with missing essential data for clustering/calculations
df_clean = df.dropna(subset=required_cols_logic).copy()
print(f"Processing {len(df_clean)} wells after removing rows with missing essential data ({required_cols_logic}).")

# Convert coordinates and key numeric columns to numeric, coercing errors
for col in required_cols_logic:
     df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows where conversion failed for essential columns
initial_count = len(df_clean)
df_clean = df_clean.dropna(subset=required_cols_logic)
if len(df_clean) < initial_count:
     print(f"Removed {initial_count - len(df_clean)} additional rows due to non-numeric values in essential columns.")

if len(df_clean) < MIN_WELLS_IN_CLUSTER:
    print(f"Not enough valid wells ({len(df_clean)}) to form a cluster of size >= {MIN_WELLS_IN_CLUSTER}. Exiting.")
    exit()

# Convert lat/lon to radians for DBSCAN haversine metric
df_clean['lat_rad'] = np.radians(df_clean[LAT_COL])
df_clean['lon_rad'] = np.radians(df_clean[LON_COL])
coords_rad = df_clean[['lat_rad', 'lon_rad']].values

print(f"Running DBSCAN with eps={EPS_KM:.2f} km ({eps_rad:.6f} radians) and min_samples={MIN_WELLS_IN_CLUSTER}...")
# Run DBSCAN
# Use min_samples = MIN_WELLS_IN_CLUSTER (now 5)
db = DBSCAN(eps=eps_rad, min_samples=MIN_WELLS_IN_CLUSTER, metric='haversine', algorithm='ball_tree')
db.fit(coords_rad)

# Add cluster labels to the dataframe
df_clean['cluster_label'] = db.labels_
n_clusters = len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)
n_noise = list(db.labels_).count(-1)
print(f'Estimated number of clusters: {n_clusters}')
print(f'Estimated number of noise points: {n_noise}')

# Process valid clusters and collect data per well
results_per_well = [] # Store results here, one entry per well in a valid zone
valid_cluster_labels = sorted([label for label in set(db.labels_) if label != -1])

print("Analyzing identified clusters...")
zone_counter = 0 # Use a simple counter for zone IDs
for label in valid_cluster_labels:
    cluster_mask = (df_clean['cluster_label'] == label)
    cluster_df = df_clean[cluster_mask].copy() # Use .copy() to avoid SettingWithCopyWarning later
    num_wells = len(cluster_df)

    # Check if the cluster meets the minimum size requirement (CHANGED to >=)
    if num_wells >= MIN_WELLS_IN_CLUSTER:
        print(f"  Cluster Label {label}: Found {num_wells} wells (>= {MIN_WELLS_IN_CLUSTER}). Checking area...")

        # Calculate cluster area using UTM projection
        cluster_coords = cluster_df[[LAT_COL, LON_COL]]
        zone_area_km2 = calculate_utm_area(cluster_coords, LAT_COL, LON_COL)

        if zone_area_km2 is not None and zone_area_km2 <= MAX_CLUSTER_AREA_KM2:
            zone_id = zone_counter # Assign a sequential ID to the valid zone
            print(f"    -> Area: {zone_area_km2:.4f} km^2 (within threshold). Zone ID: {zone_id}. Calculating stats...")

            # Calculate zone-level statistics
            zone_center_latitude = cluster_df[LAT_COL].mean()
            zone_center_longitude = cluster_df[LON_COL].mean()
            zone_average_gradient = cluster_df[GRADIENT_COL].mean()
            zone_average_bht = cluster_df[BHT_COL].mean()

            # Prepare zone info to add to each well's data
            zone_info = {
                'zone_id': zone_id,
                'zone_num_wells': num_wells,
                'zone_center_latitude': zone_center_latitude,
                'zone_center_longitude': zone_center_longitude,
                'zone_area_km2': zone_area_km2,
                'zone_average_gradient': zone_average_gradient,
                'zone_average_bht': zone_average_bht
            }
            print(f"    -> Zone {zone_id} added.")

            # Add zone information to each well in this cluster
            # Iterate through the wells of the valid cluster
            for index, well_row in cluster_df.iterrows():
                # Convert the well's original data row to a dictionary
                well_data = well_row.to_dict()
                # Add the calculated zone information to this well's dictionary
                well_data.update(zone_info)
                # Append the combined dictionary to the results list
                results_per_well.append(well_data)

            zone_counter += 1 # Increment zone ID for the next valid zone

        elif zone_area_km2 is None:
             print(f"    -> Area calculation failed for Cluster Label {label}. Skipping.")
        else:
            print(f"    -> Area: {zone_area_km2:.4f} km^2 (exceeds threshold of {MAX_CLUSTER_AREA_KM2} km^2). Skipping Cluster Label {label}.")
    # else: # Optional: log clusters found but with < MIN_WELLS_IN_CLUSTER wells
       # print(f"  Cluster Label {label} has {num_wells} wells (< {MIN_WELLS_IN_CLUSTER}). Skipping.")


# Create output DataFrame from the list of well dictionaries
if results_per_well:
    output_df = pd.DataFrame(results_per_well)
    print(f"\nProcessed {len(output_df)} wells belonging to {zone_counter} valid zones.")

    # Clean up intermediate columns and reorder for clarity
    cols_to_drop = ['lat_rad', 'lon_rad', 'cluster_label']
    output_df = output_df.drop(columns=[col for col in cols_to_drop if col in output_df.columns])

    # Define desired column order (zone info first, then original columns)
    zone_info_cols = [
        'zone_id', 'zone_num_wells', 'zone_center_latitude', 'zone_center_longitude',
        'zone_area_km2', 'zone_average_gradient', 'zone_average_bht'
    ]
    # Ensure original columns don't overlap with new zone info names by mistake
    final_original_cols = [col for col in original_columns if col in output_df.columns and col not in zone_info_cols]
    final_column_order = zone_info_cols + final_original_cols

    # Reorder columns
    output_df = output_df[final_column_order]


    print(f"Saving detailed results to: {output_csv_path}")
    try:
        # Save to CSV
        output_df.to_csv(output_csv_path, index=False, encoding='utf-8')
        print("Results saved successfully.")
    except Exception as e:
        print(f"ERROR: Could not save results to CSV: {e}")
else:
    print(f"\nNo zones found meeting the specified criteria ({MIN_WELLS_IN_CLUSTER} or more wells AND area <= {MAX_CLUSTER_AREA_KM2} km^2).")



Loading data from: wells_with_bht.xlsx
Loaded 11201 wells.
Original columns: ['Name', 'Api', 'Country', 'Status', 'Well Type', 'Latitude', 'Longitude', 'True Vertical Depth (ft)', 'True Vertical Depth (km)', 'Interpolated gradient', 'BHT', 'Suitable for ']
Processing 11201 wells after removing rows with missing essential data (['Latitude', 'Longitude', 'Interpolated gradient', 'BHT']).
Running DBSCAN with eps=0.70 km (0.000110 radians) and min_samples=5...
Estimated number of clusters: 117
Estimated number of noise points: 9731
Analyzing identified clusters...
  Cluster Label 0: Found 11 wells (>= 5). Checking area...
    -> Area: 1.6721 km^2 (exceeds threshold of 1.0 km^2). Skipping Cluster Label 0.
  Cluster Label 1: Found 5 wells (>= 5). Checking area...
    -> Area: 0.3129 km^2 (within threshold). Zone ID: 0. Calculating stats...
    -> Zone 0 added.
  Cluster Label 2: Found 45 wells (>= 5). Checking area...
    -> Area: 8.8262 km^2 (exceeds threshold of 1.0 km^2). Skipping Cluster

In [4]:
output_df

,zone_id,zone_num_wells,zone_center_latitude,zone_center_longitude,zone_area_km2,zone_average_gradient,zone_average_bht,Name,Api,Country,Status,Well Type,Latitude,Longitude,True Vertical Depth (ft),True Vertical Depth (km),Interpolated gradient,BHT,Suitable for
0,0,5,31.567891,-103.287617,0.31295,24.303991,85.868642,"UNIVERSITY ""18-31"" 3",4247532401,WARD,Plugged & Abandoned,WET GAS,31.564718,-103.290527,6865.000000,2.092,24.320783,70.879079,Suitable for heat generation
1,0,5,31.567891,-103.287617,0.31295,24.303991,85.868642,UNIVERSITY 18-29 4,4247532501,WARD,Plugged & Abandoned,CONDENSATE,31.566277,-103.282852,6825.000000,2.080,24.312277,70.569536,Suitable for heat generation
2,0,5,31.567891,-103.287617,0.31295,24.303991,85.868642,"UNIV ""18-31"" 2",4247531939,WARD,Plugged & Abandoned,GAS WELL,31.566418,-103.287827,17067.000000,5.202,24.311742,146.469682,Electricity generation
3,0,5,31.567891,-103.287617,0.31295,24.303991,85.868642,UNIVERSITY 18-31 4,4247532553,WARD,Plugged & Abandoned,WET GAS,31.569286,-103.287788,6850.000000,2.088,24.296668,70.731443,Suitable for heat generation
4,0,5,31.567891,-103.287617,0.31295,24.303991,85.868642,UNIVERSITY 18-31 6,4247532765,WARD,Plugged & Abandoned,WET GAS,31.572758,-103.289093,6850.000000,2.088,24.278483,70.693472,Suitable for heat generation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,77,11,29.263829,-95.015287,0.00180,12.129166,69.632523,"HALLS BAYOU RANCH ""3"" 3",4216731259,GALVESTON,Plugged & Abandoned,WET GAS,29.263866,-95.015274,12770.990234,3.893,12.127630,67.212864,Suitable for heat generation
472,77,11,29.263829,-95.015287,0.00180,12.129166,69.632523,STATE OF TEXAS 1,4216731260,GALVESTON,Plugged & Abandoned,WET GAS,29.263834,-95.015274,12600.589844,3.841,12.127630,66.582228,Suitable for heat generation
473,77,11,29.263829,-95.015287,0.00180,12.129166,69.632523,STATE OF TEXAS 3,4216731396,GALVESTON,Plugged & Abandoned,WET GAS,29.263781,-95.015175,12645.610352,3.854,12.115432,66.692874,Suitable for heat generation
474,77,11,29.263829,-95.015287,0.00180,12.129166,69.632523,HALLS BAYOU RANCH 8R,4216731349,GALVESTON,Plugged & Abandoned,WET GAS,29.263834,-95.015167,12546.509766,3.824,12.114494,66.325823,Suitable for heat generation


In [1]:
import pandas as pd
import numpy as np

# --- Configuration ---
# Имя входного файла (результат предыдущего скрипта)
input_csv_path = 'well_clusters_detailed_analysis.csv'
# Имя выходного файла с категориями зон
output_categories_csv_path = 'zone_categories.csv'
# Имя столбца для категоризации (убедитесь, что оно точно совпадает с вашим файлом)
suitability_col = 'Suitable for '
# Имена категорий (должны точно совпадать со значениями в столбце)
cat_electricity = 'Electricity generation'
cat_heat = 'Suitable for heat generation'
cat_mixed = 'mixed'

# --- Main Script ---

print(f"Загрузка данных из: {input_csv_path}")
try:
    df = pd.read_csv(input_csv_path)
    print(f"Загружено {len(df)} записей скважин.")
except FileNotFoundError:
    print(f"ОШИБКА: Файл не найден по пути {input_csv_path}")
    print("Убедитесь, что файл существует и путь указан верно (он должен быть результатом работы предыдущего скрипта).")
    # В Jupyter можно остановить выполнение или обработать иначе
    # raise # Раскомментируйте, если хотите остановить выполнение ячейки при ошибке
    exit() # Используем exit() если запускается как скрипт
except Exception as e:
    print(f"ОШИБКА: Не удалось прочитать CSV файл: {e}")
    # raise
    exit()

# Проверка наличия необходимых столбцов
if 'zone_id' not in df.columns:
    print("ОШИБКА: Отсутствует столбец 'zone_id' во входном файле.")
    exit()
if suitability_col not in df.columns:
    print(f"ОШИБКА: Отсутствует столбец '{suitability_col}' во входном файле.")
    exit()

print(f"Категоризация зон на основе столбца '{suitability_col}'...")

# Группировка по ID зоны
grouped_zones = df.groupby('zone_id')

zone_categories = []

# Итерация по каждой зоне
for zone_id, zone_df in grouped_zones:
    # Получаем уникальные значения 'Suitable for' в этой зоне, игнорируя NaN
    unique_values = zone_df[suitability_col].dropna().unique()

    category = cat_mixed # По умолчанию 'mixed'

    if len(unique_values) == 1:
        # Если только одно уникальное значение (не NaN)
        if unique_values[0] == cat_electricity:
            category = cat_electricity
        elif unique_values[0] == cat_heat:
            category = cat_heat
        # Если одно уникальное значение, но не совпадает с заданными,
        # оно останется 'mixed' (как и было по умолчанию)
    elif len(unique_values) == 0:
         # Если все значения NaN в этой зоне, считаем 'mixed'
         category = cat_mixed
    # Если уникальных значений > 1, категория остается 'mixed'

    zone_categories.append({'zone_id': zone_id, 'category': category})
    print(f"  Зона {zone_id}: Уникальные значения '{suitability_col}': {unique_values} -> Категория: {category}")

# Создание DataFrame с результатами
if zone_categories:
    categories_df = pd.DataFrame(zone_categories)
    print("\n--- Категоризация зон завершена ---")
    # Вывод результата в Jupyter
    from IPython.display import display
    display(categories_df)

    # Сохранение результата в CSV
    try:
        categories_df.to_csv(output_categories_csv_path, index=False, encoding='utf-8')
        print(f"\nКатегории зон сохранены в файл: {output_categories_csv_path}")
    except Exception as e:
        print(f"\nОШИБКА: Не удалось сохранить категории в CSV: {e}")

else:
    print("\nНе найдено зон для категоризации во входном файле.")



Загрузка данных из: well_clusters_detailed_analysis.csv
Загружено 476 записей скважин.
Категоризация зон на основе столбца 'Suitable for '...
  Зона 0: Уникальные значения 'Suitable for ': ['Suitable for heat generation' 'Electricity generation'] -> Категория: mixed
  Зона 1: Уникальные значения 'Suitable for ': ['Suitable for heat generation' 'Electricity generation'] -> Категория: mixed
  Зона 2: Уникальные значения 'Suitable for ': ['Suitable for heat generation' 'Electricity generation'] -> Категория: mixed
  Зона 3: Уникальные значения 'Suitable for ': ['Suitable for heat generation' 'Electricity generation'] -> Категория: mixed
  Зона 4: Уникальные значения 'Suitable for ': ['Suitable for heat generation' 'Electricity generation'] -> Категория: mixed
  Зона 5: Уникальные значения 'Suitable for ': ['Suitable for heat generation' 'Electricity generation'] -> Категория: mixed
  Зона 6: Уникальные значения 'Suitable for ': ['Electricity generation'] -> Категория: Electricity generati

,zone_id,category
0,0,mixed
1,1,mixed
2,2,mixed
3,3,mixed
4,4,mixed
...,...,...
73,73,Suitable for heat generation
74,74,Suitable for heat generation
75,75,Suitable for heat generation
76,76,Suitable for heat generation



Категории зон сохранены в файл: zone_categories.csv
